In [1]:
#importando bibliotecas
import pandas as pd
import numpy as np
import glob
import os
import sqlite3

In [4]:
#dicionario vazio.
dataSet ={}

#criando função para ler os arquivos.
def lerDados(pasta):
    arquivoCsv = glob.glob(os.path.join(pasta, '*.csv'))

    for item in arquivoCsv:
      print(item)
      nomeArquivo = os.path.splitext(os.path.basename(item))[0]
      dados = pd.read_csv(item, sep=',', encoding ='latin1')
      dataSet[nomeArquivo] = dados


lerDados('../dataSet')

../dataSet\olist_customers_dataset.csv
../dataSet\olist_geolocation_dataset.csv
../dataSet\olist_orders_dataset.csv
../dataSet\olist_order_items_dataset.csv
../dataSet\olist_order_payments_dataset.csv
../dataSet\olist_order_reviews_dataset.csv
../dataSet\olist_products_dataset.csv
../dataSet\olist_sellers_dataset.csv
../dataSet\product_category_name_translation.csv


In [5]:
#tratando nome de colunas com caractere especial (ï»¿)
for item in dataSet.values():
    item.columns = item.columns.str.replace('ï»¿', '', regex=False)

In [6]:
#tratando dados duplicados.
def removeDuplicado(dados):
    for sItem in dados.keys():
        if dados[sItem].duplicated().sum() > 0:
           print(f"DataSet {sItem} tem {dados[sItem].duplicated().sum()} dados duplicados!")
           dados[sItem] = dados[sItem].drop_duplicates()

removeDuplicado(dataSet)

DataSet olist_geolocation_dataset tem 261831 dados duplicados!


##### Limpeza e tratamentos.

In [5]:
#função responsavel por tratar valores Nulos quando o campo for tipo DATA.
def trataDataNull(dados, sAnalisado):
    try:
        pd.to_datetime(sAnalisado, format ="%Y-%m-%d %H:%M:%S", errors = "raise")
        dados = pd.to_datetime(dados, errors = "coerce")
        dados = dados.fillna(pd.to_datetime("1800-01-01 00:00:00"))
        #print(f"DEBUG--> aplicado a inserção de dados para o campo no formato DATA, 1800-01-01 00:00:00")
    except ValueError:
        #caso nao for o tipo do dado que queremos ele não altera, apenas segue em frente.
        pass
    return dados

#função responsavel por tratar valores Nulos quando o campo for tipo TEXTO.
def trataComentarioNull(dados, nmcoluna, sAnalisado):
     #veriricando se há um caracter do tipo texto "comentarios".
    if any(c.isalpha() for c in sAnalisado):
        dados = dados.fillna("sem comentarios" if "coment" in nmcoluna.lower() else "sem descricao")
        #print(f"DEBUG--> aplicado a inserção de dados para o campo no formato TEXTO, 'sem comentarios'")
    return dados

#função responsavel por tratar valores Nulos quando o campo for tipo INTEIRO ou FLOAT.
def trataValorNull(dados, sAnalisado):
    try:
        #transformando a info em float
        float(sAnalisado)   
        dados = pd.to_numeric(dados, errors = "coerce")
        dados = dados.fillna(0.00 if '.' in sAnalisado else 0)
        #print(f"DEBUG--> aplicado a inserção de dados para o campo no formato INTEIRO OU FLOAT, 0")
    except ValueError:
        pass
    return dados


def trataColunaNull(dados):
    for sNull in dados.keys():
        if dados[sNull].isnull().sum().sum() > 0:
            print(f"DataSet: {sNull}")
            for item in (dados[sNull].columns[dados[sNull].isnull().any()]):
                print(f"coluna: {item} tem um total de {dados[sNull][item].isnull().sum().sum()} registros como null!")
                #aqui vamos remover os caracteres null e converter em string os campos.
                analisar = dados[sNull][item].dropna().astype(str).head(10)

                #verifica se os campos estão todos vazios ou preenchidos, se tiver vamos pular!
                if analisar.empty:
                    print(f"DEBUG--> A coluna {item} há dados somente vazias, desconsiderar!")
                    continue

                #pega o primeiro registro e remove os espaços.
                sAnalisado = analisar.iloc[0].strip()
                
                dados[sNull][item] = trataComentarioNull(dados[sNull][item], item, sAnalisado)
                dados[sNull][item] = trataDataNull(dados[sNull][item], sAnalisado)                
                dados[sNull][item] = trataValorNull(dados[sNull][item], sAnalisado)
            print("==================================================================")

#chama funções para tratamentos.
trataColunaNull(dataSet)


DataSet: olist_orders_dataset
coluna: order_approved_at tem um total de 160 registros como null!
coluna: order_delivered_carrier_date tem um total de 1783 registros como null!
coluna: order_delivered_customer_date tem um total de 2965 registros como null!
DataSet: olist_order_reviews_dataset
coluna: review_comment_title tem um total de 87656 registros como null!
coluna: review_comment_message tem um total de 58247 registros como null!
DataSet: olist_products_dataset
coluna: product_category_name tem um total de 610 registros como null!
coluna: product_name_lenght tem um total de 610 registros como null!
coluna: product_description_lenght tem um total de 610 registros como null!
coluna: product_photos_qty tem um total de 610 registros como null!
coluna: product_weight_g tem um total de 2 registros como null!
coluna: product_length_cm tem um total de 2 registros como null!
coluna: product_height_cm tem um total de 2 registros como null!
coluna: product_width_cm tem um total de 2 registro

##### Modelo relacional.

In [6]:
#pedido do cliente
dfClientePedido = dataSet['olist_orders_dataset'].merge(dataSet['olist_customers_dataset'], on="customer_id", how="left")

#produtos e pedido de itens
dfPedidoItens = dataSet['olist_order_items_dataset'].merge(dataSet['olist_orders_dataset'], on="order_id", how="left").merge(
    dataSet['olist_products_dataset'], on="product_id", how="left")

#traducao da categoria e produtos
dfCategoriaProdutos = dataSet['olist_products_dataset'].merge(dataSet['product_category_name_translation'], on="product_category_name", how="left")

#avaliação dos pagamentos e pedidos.
dfPagamentoAvaliacao = dataSet['olist_orders_dataset'].merge(dataSet['olist_order_payments_dataset'], on="order_id", how="left").merge(
    dataSet['olist_order_reviews_dataset'], on="order_id", how="left")

#localizacao dos vendedores e itens
dfLocalizacaoVendas = dataSet['olist_order_items_dataset'].merge(dataSet['olist_sellers_dataset'], on="seller_id", how="left").merge(
    dataSet['olist_geolocation_dataset'],
    left_on="seller_zip_code_prefix",
    right_on="geolocation_zip_code_prefix",
    how="left"
)

#cliente e localização
dfGeolocalizacaoClientes = dataSet['olist_customers_dataset'].merge(
    dataSet['olist_geolocation_dataset'],
    left_on="customer_zip_code_prefix",
    right_on="geolocation_zip_code_prefix",
    how="left"
)

#pedidos, itens e pagamento
dfPagamentoItens = dataSet['olist_orders_dataset'].merge(dataSet['olist_order_items_dataset'], on="order_id", how="left").merge(
    dataSet['olist_order_payments_dataset'], on="order_id", how="left"
)

#vendedores e localização
dfLocalizacaoVendedor = dataSet['olist_sellers_dataset'].merge(
    dataSet['olist_geolocation_dataset'],
    left_on="seller_zip_code_prefix",
    right_on="geolocation_zip_code_prefix",
    how="left"
)

##### Qual o volume de pedidos por mês? Existe sazonalidade nas vendas?

In [7]:
#dfClientePedido.head(15)
pedidoVolume = (
    dfClientePedido.groupby(dfClientePedido['order_purchase_timestamp'].str[:7])['order_id']
    .nunique().reset_index(name='qtdPedidos').rename(columns={'order_purchase_timestamp': 'ano_mes'}).sort_values(by='ano_mes', ascending = False)
)

#aqui vamos calcular a diferença em porcentual entre os meses.
pedidoVolume['percent'] = pedidoVolume['qtdPedidos'].pct_change() * 100

#verificamos se a variação é superior a 20%, dessa forma preenchemos sim ou não na coluna sazonalidade.
pedidoVolume['sazonalidade'] = pedidoVolume['percent'].apply(
    lambda x: 'Sim' if abs(x) > 20 else 'Não'
)

pedidoVolume


,ano_mes,qtdPedidos,percent,sazonalidade
24,2018-10,4,NaN,Não
23,2018-09,16,300.000000,Sim
22,2018-08,6512,40600.000000,Sim
21,2018-07,6292,-3.378378,Não
20,2018-06,6167,-1.986650,Não
19,2018-05,6873,11.448030,Não
18,2018-04,6939,0.960279,Não
17,2018-03,7211,3.919873,Não
16,2018-02,6728,-6.698100,Não
15,2018-01,7269,8.041023,Não


##### Qual a distribuição do tempo de entrega dos pedidos?

In [8]:
#criando uma conexão com SQLite em memória
conn = sqlite3.connect(':memory:')

#realizando o envio do dataframe.
dfClientePedido.to_sql('dfClientePedido', conn, index=False, if_exists='replace')

# 3. Escreva e execute sua consulta
query = """
SELECT 
    order_id,
    order_purchase_timestamp as dataPedido,
    order_delivered_customer_date as dataEntrega,
    (CASE
        WHEN CAST(JULIANDAY(order_delivered_customer_date) - JULIANDAY(order_purchase_timestamp) AS INTEGER) > 0 THEN
            CAST(CAST(JULIANDAY(order_delivered_customer_date) - JULIANDAY(order_purchase_timestamp) AS INTEGER) AS TEXT) || ' dias e ' ||
            CAST((JULIANDAY(order_delivered_customer_date) - JULIANDAY(order_purchase_timestamp) - 
                 CAST(JULIANDAY(order_delivered_customer_date) - JULIANDAY(order_purchase_timestamp) AS INTEGER)) * 24 AS INTEGER) || ' horas e ' ||
            CAST((((JULIANDAY(order_delivered_customer_date) - JULIANDAY(order_purchase_timestamp)) * 24) % 1) * 60 AS INTEGER) || ' minutos foi o tempo de entrega'
        ELSE
            CAST((JULIANDAY(order_delivered_customer_date) - JULIANDAY(order_purchase_timestamp)) * 24 AS INTEGER) || ' horas e ' ||
            CAST((((JULIANDAY(order_delivered_customer_date) - JULIANDAY(order_purchase_timestamp)) * 24) % 1) * 60 AS INTEGER) || ' minutos foi o tempo de entrega'
    END) AS tempodeentrega
FROM dfClientePedido WHERE order_delivered_customer_date IS NOT NULL
"""

display(pd.read_sql_query(query, conn))


,order_id,dataPedido,dataEntrega,tempodeentrega
0,e481f51cbdc54678b7cc49136f2d6af7,2017-10-02 10:56:33,2017-10-10 21:25:13,8 dias e 10 horas e 0 minutos foi o tempo de e...
1,53cdb2fc8bc7dce0b6741e2150273451,2018-07-24 20:41:37,2018-08-07 15:27:45,13 dias e 18 horas e 0 minutos foi o tempo de ...
2,47770eb9100c2d0c44946d9cf07ec65d,2018-08-08 08:38:49,2018-08-17 18:06:29,9 dias e 9 horas e 0 minutos foi o tempo de en...
3,949d5b44dbf5de918fe9c16f97b45f8a,2017-11-18 19:28:06,2017-12-02 00:28:42,13 dias e 5 horas e 0 minutos foi o tempo de e...
4,ad21c59c0840e6cb83a9ceb5573f8159,2018-02-13 21:18:39,2018-02-16 18:17:02,2 dias e 20 horas e 0 minutos foi o tempo de e...
...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,2017-03-09 09:54:05,2017-03-17 15:08:01,8 dias e 5 horas e 0 minutos foi o tempo de en...
99437,63943bddc261676b46f01ca7ac2f7bd8,2018-02-06 12:58:58,2018-02-28 17:37:56,22 dias e 4 horas e 0 minutos foi o tempo de e...
99438,83c1379a015df1e13d02aae0204711ab,2017-08-27 14:46:43,2017-09-21 11:24:17,24 dias e 20 horas e 0 minutos foi o tempo de ...
99439,11c177c8e97725db2631073c19f07b62,2018-01-08 21:28:27,2018-01-25 23:32:54,17 dias e 2 horas e 0 minutos foi o tempo de e...
